## 2.4 Model Selection 소개
### 학습/테스트 데이터 셋 분리 – train_test_split()

In [4]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# train, test 데이터 구분 없이 넣은 잘못된 예 
iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target
dt_clf.fit(train_data, train_label)

# 학습 데이터 셋으로 예측 수행
pred = dt_clf.predict(train_data)
print('예측 정확도:',accuracy_score(train_label,pred))

# 반드시 별도로 test데이터를 만들어야 되는데
# 이 test데이터를 만드는 것이 굉장히 중요
# train데이터와 너무 비슷하게 만들어버리면, 알고리즘이 안좋음에도(적합하지 않음에도) 불구하고 
# 이미 비슷한 학습데이터로 학습을 한 것이기 때문에 결과가 좋게 나올 수밖에 없음 
# 알고리즘을 얼마나 잘 학습을 했는지의 판단은
# 기존의 학습데이터에 나와있지 않는 데이터들에 대해서도 얼마나 잘 예측할 수 있는 지와 밀접한 관련이 있음  

예측 정확도: 1.0


In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

dt_clf = DecisionTreeClassifier( )
iris_data = load_iris()

X_train, X_test,y_train, y_test= train_test_split(iris_data.data, iris_data.target, 
                                                    test_size=0.3, random_state=121)

In [7]:
dt_clf.fit(X_train, y_train) # 학습데이터(피처,라벨)로 dt_clf에 학습이 됨. 
pred = dt_clf.predict(X_test) # 학습된 dt_clf(estimator)에 테스트데이터(피처)를 넣고 라벨 예측
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))#테스트 라벨값의 실제값과 예측값을 넣고 정확도 구하기

예측 정확도: 0.9556


넘파이 ndarray 뿐만 아니라 판다스 DataFrame/Series도 train_test_split( )으로 분할 가능

In [8]:
import pandas as pd

iris_df = pd.DataFrame(iris_data.data, columns=iris_data.feature_names)
iris_df['target']=iris_data.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [9]:
ftr_df = iris_df.iloc[:, :-1]
tgt_df = iris_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(ftr_df, tgt_df, 
                                                    test_size=0.3, random_state=121)

In [10]:
print(type(X_train), type(X_test), type(y_train), type(y_test))

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [11]:
dt_clf = DecisionTreeClassifier( )
dt_clf.fit(X_train, y_train) # ndarray뿐만 아니라 dataframe도 fit(), predict() 에 들어갈 수 있음
pred = dt_clf.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred))) #accuracy_score()에도 dataframe 들어갈 수 있음


예측 정확도: 0.9556


### 교차 검증
#### K 폴드 

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 폴드 세트로 분리하는 KFold 객체(kfold)와 폴드 세트별 정확도를 담을 리스트 객체(cv_accuracy)생성.
kfold = KFold(n_splits=5)
cv_accuracy = []
print('붓꽃 데이터 세트 크기:',features.shape[0]) # 행의 수 150이 나옴
# 이 150을 5개의 폴드세트로 나누기 때문에 학습데이터세트는 120개, 검증용테스트의 데이터는 30개가 됨
#(원래 학습-테스트 나누고 학습에서 학습-검증을 나누는데, 여기서는 학습-검증으로만 나눠서 KFold 예제를 보여줌 )

붓꽃 데이터 세트 크기: 150


In [15]:
n_iter = 0

# KFold객체의 split( ) 호출하면 폴드 별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환  
# (kfold.split(features)은 학습용 ndarray를 가지는 위치인덱스, 검증용테스트 데이터의 위치인덱스를 반환한다.)
# 5번 수행해야 해서 이 loop를 5번 돌 것이다. 
# 'kfold.split(features)'가 반환하는 것이 사실은 generator라는 것 
for train_index, test_index  in kfold.split(features): 
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    #학습 및 예측 
    dt_clf.fit(X_train , y_train)    
    pred = dt_clf.predict(X_test)
    n_iter += 1
    
    # 반복 시 마다 정확도 측정 
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    
    cv_accuracy.append(accuracy)
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산 
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy)) 


#1 교차 검증 정확도 :1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 :0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#2 검증 세트 인덱스:[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 :0.8667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#3 검증 세트 인덱스:[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 :0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#4 검증 세트 인덱스:[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 :0.7333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#5 검증 세트 인덱스:[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.9


#### Stratified K 폴드  
분류는 명확하게 Stratified K폴드를 사용해야 하는데, 이게 조금 수행성능이 느리다 보니, 가끔 일반 K폴드를 쓰는 경우도 있음  
(일반K폴드를 사용한다면 주의할점은, 학습과 검증의 데이터세트의 레이블값의 분포도가 유사해야 함!!!)

In [19]:
import pandas as pd

iris = load_iris()

iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label']=iris.target
iris_df['label'].value_counts() # 레이블값의 분포가 0,1,2 각 값마다 50개씩 똑같이 들어있음

2    50
1    50
0    50
Name: label, dtype: int64

In [20]:
kfold = KFold(n_splits=3)
# kfold.split(전체데이터)는 폴드 세트를 3번 반복할 때마다 달라지는 학습/테스트 용 데이터 로우 인덱스 번호 반환. 
n_iter =0
for train_index, test_index  in kfold.split(iris_df):
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())
# 결과를 보면 학습과 검증 레이블 데이터의 분포가 이상함...

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64


In [23]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)
n_iter=0

for train_index, test_index in skf.split(iris_df, iris_df['label']): #일반K폴드와 다른점이 lable값 데이터도 넣어줌
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차 검증: 1
학습 레이블 데이터 분포:
 2    40
1    40
0    40
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    10
1    10
0    10
Name: label, dtype: int64
## 교차 검증: 2
학습 레이블 데이터 분포:
 2    40
1    40
0    40
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    10
1    10
0    10
Name: label, dtype: int64
## 교차 검증: 3
학습 레이블 데이터 분포:
 2    40
1    40
0    40
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    10
1    10
0    10
Name: label, dtype: int64
## 교차 검증: 4
학습 레이블 데이터 분포:
 2    40
1    40
0    40
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    10
1    10
0    10
Name: label, dtype: int64
## 교차 검증: 5
학습 레이블 데이터 분포:
 2    40
1    40
0    40
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    10
1    10
0    10
Name: label, dtype: int64


In [24]:
dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter=0
cv_accuracy=[]

# StratifiedKFold의 split( ) 호출시 반드시 레이블 데이터 셋도 추가 입력 필요  
for train_index, test_index  in skfold.split(features, label):
    # split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    #학습 및 예측 
    dt_clf.fit(X_train , y_train)    
    pred = dt_clf.predict(X_test)

    # 반복 시 마다 정확도 측정 
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
    
# 교차 검증별 정확도 및 평균 정확도 계산 
print('\n## 교차 검증별 정확도:', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도:', np.mean(cv_accuracy)) 


#1 교차 검증 정확도 :0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#1 검증 세트 인덱스:[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차 검증 정확도 :0.94, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#2 검증 세트 인덱스:[ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차 검증 정확도 :0.98, 학습 데이터 크기: 100, 검증 데이터 크기: 50
#3 검증 세트 인덱스:[ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도: [0.98 0.94 0.98]
## 평균 검증 정확도: 0.9666666666666667


#### cross_val_score( )  
교차검증을 보다 간편하게 하는 방법(함수 하나로 폴드세트 추출, 학습/예측, 평가를 한번에 수행)

In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score , cross_validate
from sklearn.datasets import load_iris
import numpy as np

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

feature = iris_data.data
label = iris_data.target

# 성능 지표는 정확도(accuracy) , 교차 검증 세트는 3개 
scores = cross_val_score(dt_clf , feature , label , scoring='accuracy',cv=3)# 반환값은 cv의 수만큼 정확도 지표가 나옴
#(cross_val+score() 인자 estimator가 '분류'일때는 StratifiedKFold로, '회귀'일 때는, KFold로 자동적으로 해줌 )
print('교차 검증별 정확도:',np.round(scores, 4)) # 3개의 정확도 지표(accuracy)가 나옴 
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도: 0.9667


#### GridSearchCV  
GridSearchCV를 이용하여 Classifier나 Regressor과 같은 알고리즘에 사용되는 하이퍼 파라미터를 순차적으로 입력하면서 편리하게 최적의 파라미터를 도출할 수 있는 방안을 제공함

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# 데이터를 로딩하고 학습데이터와 테스트 데이터 분리
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, 
                                                    test_size=0.2, random_state=121)
dtree = DecisionTreeClassifier()
# 여기에서는 결정트리 알고리즘을 좀더 튜닝을 하고 싶어서 하이퍼 파라미터를 순차적으로 접근하면서
# 최적의 파라미터를 도출할 수 있는 방안을 제공 
### parameter 들을 dictionary 형태로 설정
parameters = {'max_depth':[1, 2, 3], 'min_samples_split':[2,3]} #단일값이어도 리스트형태로 [] 해줘야 함

In [3]:
import pandas as pd

# param_grid의 하이퍼 파라미터들을 3개의 train, test set fold 로 나누어서 테스트 수행 설정.  
### refit=True 가 default 임. True이면 가장 좋은 파라미터 설정으로 재 학습 시킴.
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True, return_train_score=True)

# 붓꽃 Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가 .
grid_dtree.fit(X_train, y_train) # 6*3=18번 수행하게 됨 (6: 파라미터 3*2/ 3: cv수)

# refit=True의 의미는 GridSearchCV가 학습하고 검증하면서 가장 좋은 파라미터 값을 찾아내면, 
# 그 하이퍼파라미터로 최종적으로 estimator로 입력된 객체(dtree)에 학습을 시켜버림. 
# 즉, fit을 호출해버려서 dtree객체를 학습을 시켜버린다는 것.
# return_train_score(train score을 가져오기 위해서 쓰는건데)은 그렇게 중요한 것 아님. 

# GridSearchCV 결과는 cv_results_ 라는 딕셔너리로 저장됨. 이를 DataFrame으로 변환(cv_results_ 딕셔너리가 번잡하기 때문에)
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [ ]:
grid_dtree.cv_results_ #딕셔너리 형태인데, 엄청 번잡함-(그래서 위에서 Dataframe으로 변환하고 원하는 컬럼만 추출해서 본 것)

In [30]:
print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dtree.best_score_))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.9750


In [28]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator 반환
estimator = grid_dtree.best_estimator_  # refit으로 이미 (최적의 파라미터로)학습이 된 estimator인 'dtree'를 나타냄

# GridSearchCV의 best_estimator_는 이미 최적 하이퍼 파라미터로 학습이 됨
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

테스트 데이터 세트 정확도: 0.9667


In [29]:
# refit=True로 설정된 GridSearchCV 객체가 fit()을 수행 시 학습이 완료된 Estimator('dtree')를 내포하고 있으므로 predict()를 통해 예측도 가능. 
pred = grid_dtree.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

테스트 데이터 세트 정확도: 0.9667


**-위의 estimator.predict(X_test)와 grid_dtree.predict(X_test) 결과가 같은데, estimator와 grid_dtree의 차이?**  
grid_dtree는 Estimator객체가 아닌 GridSearchCV 객체 입니다.  
GridSearchCV는 생성자로 Estimator객체와 하이퍼 파라미터 튜닝을 수행할 파라미터들, 그리고 cv 횟수등을 생성 인자로 받습니다.  
GridSearchCV는 본질적으로는 교차검증을 위한 하이퍼 파라미터 튜닝 방법을 제공하는 클래스이지만 Estimator(Regressor, Classifier)와 유사하게 fit()과 predict() 메소드를 가지고 있습니다.  
GridSearchCV의 fit()는 인자로 피처 데이터 세트, 타겟 데이터 세트를 받으며, GridSearchCV의 생성자로 입력받은 Estimator객체를 입력 받은 하이퍼 파라미터들을 번갈아 가면서 fit()인자로 입력받은 데이터 세트에 대해 학습과 검증용 데이터 폴드 세트로 나누면서 최적의 하이퍼 파라미터를 찾습니다. 이때 만일 GridSearchCV의 생성자로 refit=True(디폴트가 refit=True 입니다)를 입력하게 되면 GridSearchCV에서 사용된 Estimator객체를 최적 하이퍼 파라미터로 최종적으로 학습 시킨 뒤 이를 best_estimator_ 객체 속성으로 가지고 있게 됩니다.  
GridSearchCV는 predict() 메소드 역시 가지고 있습니다. 이 predict()메소드를 수행하게 되면 GridSearchCV 객체내의 best_estimator_ 객체, 즉 GridSearchCV의 생성자로 입력되어 최적의 하이퍼 파라미터로 학습된 Estimator의 predict() 메소드를 호출하는 것입니다. GridSearchCV의 predict()는 단순히 이 역할만 하는 것입니다.  

소스코드에서 estimator = grid_dtree.best_estimator_ 는 결국은 최종 하이퍼 파라미터로 학습된 결정트리 dtree 객체를 의미하며, grid_dtree.predict(X_test)는 결국 estimator.predict(X_test)와 동일합니다. 

**-GridSearch관련 추가 자세한 설명**

사이킷런은 GridSearchCV 라는 객체를 제공합니다. GridSearchCV는 하이퍼 파라미터들을 순차적으로 대입해 가면서 모델을 학습 시키고 검증해서 최적 하이퍼 파라미터를 찾는데, 이때 Cross validation (교차 검증) 방식으로 학습과 검증을 합니다.  
GridSearchCV() 생성 시 입력 인자로 사이킷런 Decision Tree와 같은 Estimator객체와 테스트할 하이퍼 파라미터들을 Dictionary 형태로 입력해주면 자동으로 Decision Tree를 입력된 하이퍼 파라미터들로 반복적으로 학습 후 성능 검증을 합니다.  

#DecisionTree 객체 생성.  
dt_clf = DecisionTreeClassifier()  

#테스트할 하이퍼 파라미터 설정. max_depth를 4, 5, 6 값을 번갈아서 테스트  
hyper_params = {'max_depth': [ 4, 5, 6] }  

#GridSearchCV 객체 생성 
grid_cv = GridSearchCV(dt_clf, param_grid=hyper_params, cv=3, refit=True)  

#학습데이터 x_train, y_train에 대해서 GridSearchCV  수행.  max_depth 4일때 모델 학습, 성능 검증, 5일때 학습 및 성능 검증, 6일때 학습 및 성능 검증을 하는데 교차 검증을 3으로 하므로 3개의 max_depth 하이퍼 파라미터 각각에 대해서 3번씩 학습, 검증 수행하므로 3x3번 학습, 3x3번 검증, 총 9번의 학습 검증을 수행  
grid_cv.fit(x_train, y_train)  

학습과 예측 모두 GridSearchCV에서 입력된 DecisionTreeClassifier 객체를 이용해서 수행합니다.  
GridSearchCV는 해당 DecisionTreeClassifier 객체를 GridSearchCV에 입력된 하이퍼 파라미터와 CV값에 따라 반복적으로 학습 시키고 성능 검증하고, 그 결과를 별도의 Dictionary에 기록하는 역할을 수행합니다.(grid_cv.cv_results_)  
최종적으로 가장 좋은 성능을 나타낸 하이퍼 파라미터로 DecisionTreeClassifier 객체를 최종 학습 시키고 종료 합니다.  
GridSearchCV에서 사용된 Estimator객체를 최적 하이퍼 파라미터로 최종적으로 학습 시킨 뒤 이를 best_estimator_ 객체 속성으로 가지고 있게 됩니다.  
+추가)  
GridSearchCV의 fit()는 인자로 피처 데이터 세트, 타겟 데이터 세트를 받으며, GridSearchCV의 생성자로 입력받은 Estimator객체를 입력 받은 하이퍼 파라미터들을 번갈아 가면서 fit()인자로 입력받은 데이터 세트에 대해 학습과 검증용 데이터 폴드 세트로 나누면서 최적의 하이퍼 파라미터를 찾습니다. 이때 만일 GridSearchCV의 생성자로 refit=True(디폴트가 refit=True 입니다)를 입력하게 되면 GridSearchCV에서 사용된 Estimator객체를 최적 하이퍼 파라미터로 최종적으로 학습 시킨 뒤 이를 best_estimator_ 객체 속성으로 가지고 있게 됩니다.  
GridSearchCV는 predict() 메소드 역시 가지고 있습니다. 이 predict()메소드를 수행하게 되면 GridSearchCV 객체내의 best_estimator_ 객체, 즉 GridSearchCV의 생성자로 입력되어 최적의 하이퍼 파라미터로 학습된 Estimator의 predict() 메소드를 호출하는 것입니다. GridSearchCV의 predict()는 단순히 이 역할만 하는 것입니다.  

**-위에서 grid_dtree.best_score_값과 accuracy_score(y_test,pred) 값이 다른 이유**  
GridSearchCV 최고 정확도 grid_dtree.best_score_값은 0.975이고,  
테스트 데이터 세트 정확도 grid_dtree.predict(X_test)해서 나온 accuracy_score(y_test,pred) 값은 0.9667이다.  
-다른 이유는?  
위의 소스코드 처음부터 보면,  X_train, X_test, y_train, y_test= train_test_split(..., test_size=0.2)로  학습 데이터 세트(80%)와 테스트 데이트 세트(20%)가 분리되었습니다.  
grid_dtree = GridSearch(... cv=3)  
grid_dtree.fit(X_train, y_train)으로 학습 데이터 세트가 입력되면 GridSearchCV는 주어진 X_train, y_train으로 3개 Fold로 다시 학습/Valid 를 나눕니다. 즉 iris 데이터의 80%인 X_train을 다시 분할해서 학습(X_train의 2/3)과 Valid(X_train의 1/3)으로 나누면서 cross validation으로 학습과 검증을 수행하게 됩니다. 즉 X_train을 쪼개면서 학습하고 predict()한 최고 정확도를 grid_dtree.best_score_ 가 가지는 것입니다.  
하지만 이는 테스트 데이터 X_test로 predict() 된게 아닙니다. 그렇기 때문에 X_test 로 predict()된 결과와 다릅니다.

### '질문답변' 관련 추가설명 정리

#### 3가지 API 요약 정리
개별 API는 각각의 용도가 있습니다.  
-train_test_split()는 간편하게 전체 데이터를 학습/테스트(또는 검증)용 데이터 세트로 분리하는 데 사용됩니다.  
-cross_val_score()는 간편하게 교차검증 결과 평가 지표(정확도, 정밀도등)를 반환해 줍니다.  
-GridSearchCV는 교차 검증을 통해 최적 하이퍼 파라미터를 찾아주는 역할을 합니다.  
개별 용도에 맞게 각각을 사용하시면 됩니다.  

#### cross_val_score() 에서 cv의 방법은? (회귀-KFold/분류-StratifiedFold)
-데이터를 '폴드' 방식으로 교차 검증 데이터 셋을 만들때, KFold와 StratifiedKFold를 이용합니다.  
-cross_val_score는 폴드 교차 검증 데이터 셋을 '분류'일때는 기본적으로 StratifiedKFold를 이용합니다.  
단, '회귀'일때는 KFold를 이용하는 데, 이는 '회귀'의 타겟값은 '분류'의 타겟값과 다르게 '특정 이산값'이 아니라 '연속값'이기 때문에 Stratified 기반으로 Fold를 만들 수 없기 때문입니다.  
-일반적으로 학습 데이터와 테스트 데이터를 단순 분리할 때는, 사이킷런에서 train_test_split()을 이용합니다. 이 때 Stratified 방식과 같이 학습과 테스트 데이터의 타겟값 비율을 맞추기를 원한다면 함수 인자로 train_test_split(...., stratify=타겟값) 를 입력하면 됩니다.

#### GridSearchCV에서 cv의 방법은? (회귀-KFold/분류-StratifiedFold)
GridSearchCV는 기본적으로 분류(Classification)일 경우 Stratified K 방식입니다. 회귀에서는 Stratified K 를 쓸수 없기에 K Fold입니다. 더 자세한 내용은,  
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html  
에서 cv parameter 설명에서 확인할 수 있음

#### '검증'을 하는 의미?
검증을 하는 이유는 말 그대로 해당 데이터가 여러번 성능 평가 검증을 수행하는 것입니다. 한번 시험보지 않고, 여러번 시험을 보는 것입니다.  다만 테스트 데이터로 검증을 하지 않는 이유는 모델을 여러번 테스트/검증하고 하이퍼 파라미터도 튜닝해보면서 완전하게 성능이 올라온다고 판단될 경우 그동안 오픈 되지 않는 데이터 세트로 테스트를 해보는 것입니다.  
너무 교차 검증과 테스트 데이터에 대한 개념적인 사항에 집중하실 필요는 없습니다. 실전에서는 교차 검증으로 여러번 테스트 하고, 가능하면, 테스트 데이터는 모델이 튜닝/성숙이 완료 될 때까지 기다린 다음에 최종 테스트를 해보기 위해 숨겨놓는 것 뿐입니다. 

#### GridSearchCV에서 하이퍼 파라미터 지정 관련
-질문: max_depth는 1,2,3 / min_samples_split은 2,3으로 지정했는데,이걸 1-10 / 2-10 이런식으로 개수를 늘리게 되면 더 최적의 하이퍼파라미터를 찾을 수 있지만 시간이 오래걸려서 그렇게 하지 않는건가요? (어떻게 결정하는건지?)  
-답변)네, 맞습니다.GridSearchCV는 그런 단점이 있습니다. 너무 많은 파라미터값을 입력하면 시간이 너무 오래 걸립니다.  
어떤 수를 넣을지는 답이 없습니다. 알고리즘 별로, 데이터 유형에 따라(피처 갯수가 많다던가) 개략적인 하이퍼파라미터를 설정하여 입력합니다. 가령 DecisionTree라면 너무 max_depth가 크지 않게 설정한다던가, 어느정도 튜닝을 해보면 감이 생기는데, 거기에 기반하여 결정합니다.